# Tarea 1: Analítica textual

## Ejemplo de uso

In [18]:
from construir_modelo import ConstruirYGuardarModelo
from cargar_modelo import (
    CargarModelo,
    PreProcesarConsulta,
    RecuperarDocumentosRelevantes,
    MostrarDocumentos
)
# Especificar el modelo a utilizar
nombre_modelo = "Modelo_V3" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = input("Ingrese query: ")

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

print(f'query:{terms}')

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=False)


query:medical engineer in ohio
Lista de documentos relevantes a la query:

Doc: doc_1576 (Similitud: 0.3781)
Doc: doc_1575 (Similitud: 0.3781)
Doc: doc_1579 (Similitud: 0.3751)
Doc: doc_1643 (Similitud: 0.2511)
Doc: doc_1578 (Similitud: 0.2491)
Doc: doc_3297 (Similitud: 0.2418)
Doc: doc_1652 (Similitud: 0.2278)
Doc: doc_1654 (Similitud: 0.2275)
Doc: doc_418 (Similitud: 0.2093)
Doc: doc_1729 (Similitud: 0.2082)


# **Metodología: Evaluación de versiones del Modelo**

Para evaluar la calidad de nuestro sistema de recuperación de información, utilizamos la métrica *Mean Reciprocal Rank* (MRR), que se define como:

$$
MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{\text{rank}_i}
$$

Donde:
- $|Q|$ es el número total de consultas o perfiles evaluados.
- $\text{rank}_i$ es la posición en el ranking donde aparece el primer documento relevante para la \(i\)-ésima consulta.

### **Metodología de Evaluación** 

1. **Selección de Consultas**: Se eligen manualmente un conjunto de perfiles o consultas representativas para evaluar el modelo.
2. **Generación de Rankings**: Para cada consulta, el modelo produce una lista ordenada de documentos (descripciones de trabajo) según su relevancia.
3. **Identificación de Documentos Relevantes**: Se determina manualmente cuáles documentos son relevantes para cada consulta ( Máx. 2).
4. **Cálculo del Rank**: Se identifica la posición $\text{rank}_i$ del primer documento relevante en el ranking generado.
5. **Cálculo del MRR**: Se calcula el MRR utilizando la fórmula anterior, promediando los recíprocos de los rangos obtenidos.





# **Resultados: Evaluación de las Diferentes Versiones del Modelo**

A continuación se presenta una tabla que resume los resultados de evaluación de tres versiones del modelo utilizando la métrica MRR (Mean Reciprocal Rank). Para cada versión, se muestra la posición en la que se encontró el primer documento relevante para cada consulta y el MRR calculado.

## **Resultados de Evaluación**

| Versión  | Consulta                                                                        | Posición del Documento Relevante | MRR    |
|----------|---------------------------------------------------------------------------------|----------------------------------|--------|
| **V1**   | Software engineer with experience in Python and machine learning                | 2                                |        |
|          | Marketing manager skilled in digital marketing and SEO strategies               | 1                                |        |
|          | Data analyst proficient in Python and data visualization tools                  | 1                                |        |
|          | **MRR Calculado**                                                               |                                  | 0.8333 |
| **V2**   | Software engineer with experience in Python and machine learning                | 1                                |        |
|          | Marketing manager skilled in digital marketing and SEO strategies               | 2                                |        |
|          | Data analyst proficient in Python and data visualization tools                  | 1                                |        |
|          | **MRR Calculado**                                                               |                                  | 0.8333 |
| **V3**   | Software engineer with experience in Python and machine learning                | 1                                |        |
|          | Marketing manager skilled in digital marketing and SEO strategies               | 1                                |        |
|          | Data analyst proficient in Python and data visualization tools                  | 1                                |        |
|          | **MRR Calculado**                                                               |                                  | 1.0000 |

---

#  **Discusión: Análisis de las Mejoras en Cada Versión**

## **Versión 1**

- **Configuración del Modelo:**
  - Sin eliminación de *stopwords*.
  - Sin lematización.
  - Parámetros del `TfidfVectorizer`:
    - `min_df=1`
    - `max_df=1.0`
    - `ngram_range=(1,1)`

- **Resultados:**
  - Dos consultas encontraron el documento relevante en la posición 1.
  - Una consulta lo encontró en la posición 2.
  - **MRR de 0.8333**

- **Análisis:**
  - La ausencia de preprocesamiento avanzado limita la capacidad del modelo para capturar sinónimos y formas variantes de palabras.
  - Al no eliminar *stopwords*, el modelo puede verse afectado por ruido de palabras comunes que no aportan significado significativo.

## **Versión 2**

- **Cambios Introducidos:**
  - **Se implementó la lematización.**
  - Se mantuvo sin eliminar *stopwords*.
  - Mismos parámetros del `TfidfVectorizer` que en V1.

- **Resultados:**
  - Dos consultas encontraron el documento relevante en la posición 1.
  - Una consulta en la posición 2 (diferente a la de V1).
  - **MRR de 0.8333** (igual que en V1)

- **Análisis:**
  - La lematización ayudó al modelo a reconocer diferentes formas de la misma palabra (por ejemplo, "running" y "run"), mejorando la comprensión semántica.
  - Sin embargo, al no eliminar *stopwords*, el modelo aún puede estar influenciado por palabras comunes sin significado significativo.
  - La mejora en una consulta se compensó con una ligera disminución en otra, manteniendo el MRR igual.

## **Versión 3**

- **Cambios Introducidos:**
  - **Se implementó la eliminación de *stopwords*.**
  - **Se mantuvo la lematización.**
  - **Ajustes en los parámetros del `TfidfVectorizer`:**
    - `min_df=5` (elimina términos que aparecen en menos de 5 documentos)
    - `max_df=0.8` (elimina términos que aparecen en más del 80% de los documentos)
    - `ngram_range=(1,2)` (considera unigramas y bigramas)

- **Resultados:**
  - Todas las consultas encontraron el documento relevante en la posición 1.
  - **MRR mejoró a 1.0000**

- **Análisis:**
  - La eliminación de *stopwords* redujo el ruido en los datos, permitiendo que el modelo se enfoque en palabras más significativas.
  - La lematización continuó mejorando la capacidad del modelo para entender variantes de palabras.
  - El ajuste de los parámetros del `TfidfVectorizer`:
    - **`min_df=5`** ayudó a eliminar términos poco frecuentes que no aportan valor.
    - **`max_df=0.8`** eliminó términos demasiado comunes que podrían no ser útiles para distinguir documentos.
    - **`ngram_range=(1,2)`** permitió capturar frases de dos palabras, mejorando la comprensión de contextos específicos.
  - Estas mejoras condujeron a que los documentos relevantes fueran clasificados en la primera posición para todas las consultas, elevando el MRR al valor máximo posible de 1.0000.

---

# **Conclusiones y Recomendaciones**

- **Importancia del Preprocesamiento:**
  - La **eliminación de *stopwords*** y la **lematización** son técnicas de preprocesamiento esenciales que mejoran significativamente el rendimiento del modelo al reducir el ruido y normalizar las palabras.
  
- **Ajuste de Parámetros del Modelo:**
  - Modificar parámetros como **`min_df`**, **`max_df`** y **`ngram_range`** en el `TfidfVectorizer` puede tener un impacto considerable en la calidad de los resultados.
    - **`min_df`** ayuda a eliminar palabras demasiado raras que pueden no ser útiles.
    - **`max_df`** filtra palabras demasiado comunes que no ayudan a distinguir entre documentos.
    - **`ngram_range`** permite capturar contexto adicional mediante la consideración de secuencias de palabras (por ejemplo, bigramas).

- **Mejora Continua:**
  - Los resultados demuestran que **iterar y ajustar** tanto el preprocesamiento como los parámetros del modelo conduce a mejoras en el rendimiento.
  - Es recomendable continuar experimentando con diferentes configuraciones y técnicas avanzadas (como modelos semánticos) para seguir optimizando el modelo.

- **Implementación de Técnicas Avanzadas (Futuro Trabajo):**
  - Considerar la incorporación de modelos de lenguaje pre-entrenados como **BERT** o **Word Embeddings** para capturar relaciones semánticas más profundas.
  - Implementar métodos de **reducción de dimensionalidad** o **selección de características** para mejorar aún más el rendimiento.

---

**Nota:** La mejora del MRR de 0.8333 a 1.0000 en la **Versión 3** indica que el modelo, con las configuraciones y preprocesamiento aplicados, es capaz de recuperar consistentemente el documento relevante en la primera posición para todas las consultas evaluadas, lo cual es un resultado óptimo en términos de ranking y relevancia.


## Modelo Versión 1:


In [1]:
from cargar_modelo import CargarModelo, PreProcesarConsulta, RecuperarDocumentosRelevantes, MostrarDocumentos

# Especificar el modelo a utilizar
nombre_modelo = "Modelo_V1" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Software engineer with experience in Python and machine learning"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

# Mostrar resultados
print(f'query:{terms}')

MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)



query:Software engineer with experience in Python and machine learning
Lista de documentos relevantes a la query:

Doc: doc_3637 (Similitud: 0.5741)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: machine learning engineerposition: Machine Learning Engineer description: [' Python ? JAVA ? API ? Cloud solution ? '] location: Brussels

Doc: doc_3651 (Similitud: 0.5026)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: machine learning engineerposition: Principal Machine Learning Engineer description: [' Building production-level machine learning models for a variety of clients. Solving complex commercial problems with innovative machine learning solutions. ', ' An MSc or Ph.D. in a quantitative discipline (e.g. Maths, Stats, Computer Science, Physics etc) Experience programming with production-level Python. Experience in building machine learning models and deploying them into production. A pa

In [2]:
nombre_modelo = "Modelo_V1" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Marketing manager skilled in digital marketing and SEO strategies"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)



Lista de documentos relevantes a la query:

Doc: doc_2730 (Similitud: 0.5143)
Contenido del documento:
major_job: BUSINESS AND ADMINISTRATION PROFESSIONALS job: marketerposition: Digital Marketer description: [' Degree in a Digital Marketing related subject, with a placement of at least 6 months in a Digital Marketing role OR have at least 2 years’ work experience in a Digital Marketing role. Experience of working in an agency environment. Ability to identify areas of opportunity within digital marketing and provide advice on ways to maximise those opportunities. Ability to develop strong working relationships, internally and externally. Strong written and verbal communication skills. A willingness to take responsibility for quality and deadlines. An eye for detail, proactive and able to take the initiative. A positive attitude, with the ability to meet deadlines and deliver results under pressure. Solutions focused. ', ' Create and maintain relevant and engaging Content, SEO, Social M

In [3]:
nombre_modelo = "Modelo_V1" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)


# Ingresar la consulta
terms = "Data analyst proficient in Python and data visualization tools"



# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)



Lista de documentos relevantes a la query:

Doc: doc_3514 (Similitud: 0.4034)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: data analystposition: Senior Business Data Analyst description: [' Strong Data Analyst background ETL Relational Datamodelling Good understanding of Data Warehouse solutions Excellent stakeholder management skills ', ' Financial Services background SQL, Python '] location: London

Doc: doc_3520 (Similitud: 0.3418)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: data analystposition: Data Analyst Focus Glossary (M/F) description: [' A Data Analyst to support the continuous improvement and deployment of the Data Governance capabilities at the Bank, in order to enforce our Data Quality & Governance Team and its growing activity. '] location: Luxembourg City

Doc: doc_3518 (Similitud: 0.3030)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY 

## Resultado Modelo 1

In [15]:

# Especificar el nombre del modelo
nombre_modelo = "Modelo_V1"  

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Definir las consultas y los documentos relevantes
consultas_relevantes = {
    "Software engineer with experience in Python and machine learning": [
        "doc_3651", 
        "doc_3654"
    ],
    
    "Marketing manager skilled in digital marketing and SEO strategies": [
        "doc_4082", 
        "doc_2730"
    ],
    
    "Data analyst proficient in Python and data visualization tools": [
        "doc_3514", 
        "doc_3520"
    ]
}


# Definir la función calcular_mrr (asegúrate de tenerla en tu código)

def calcular_mrr(consultas_relevantes, vectorizer, tfidf_matrix, doc_id, remove_stopwords, use_lemmatization):
    mrr_total = 0
    num_consultas = len(consultas_relevantes)
    
    for consulta, docs_relevantes in consultas_relevantes.items():
        # Preprocesar y transformar la consulta
        query_preprocesada = PreProcesarConsulta(consulta, remove_stopwords, use_lemmatization)
        query_vector = vectorizer.transform([query_preprocesada])
        
        # Obtener similitudes coseno y rankings
        cosine_similarities = (tfidf_matrix * query_vector.T).toarray().flatten()
        top_indices = cosine_similarities.argsort()[::-1]
        ranked_docs = [(cosine_similarities[idx], doc_id[idx]) for idx in top_indices]
        
        # Encontrar el rank del primer documento relevante
        rank_relevante = None
        for idx, (sim, doc) in enumerate(ranked_docs, start=1):
            if doc in docs_relevantes:
                rank_relevante = idx
                break
        
        if rank_relevante:
            reciprocal_rank = 1 / rank_relevante
            mrr_total += reciprocal_rank
            print(f"Consulta '{consulta}': Documento relevante encontrado en la posición {rank_relevante}.")
        else:
            print(f"Consulta '{consulta}': No se encontró un documento relevante en el ranking.")
    
    # Calcular el MRR
    mrr = mrr_total / num_consultas
    print(f"\nEl MRR calculado es: {mrr:.4f}")
    return mrr

# Calcular el MRR para el modelo cargado
mrr = calcular_mrr(
    consultas_relevantes,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords,
    use_lemmatization
)

Consulta 'Software engineer with experience in Python and machine learning': Documento relevante encontrado en la posición 2.
Consulta 'Marketing manager skilled in digital marketing and SEO strategies': Documento relevante encontrado en la posición 1.
Consulta 'Data analyst proficient in Python and data visualization tools': Documento relevante encontrado en la posición 1.

El MRR calculado es: 0.8333


## Modelo Versión 2:


In [5]:
# Especificar el modelo a utilizar
nombre_modelo = "Modelo_V2" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Software engineer with experience in Python and machine learning"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

# Mostrar resultados
print(f'query:{terms}')

MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)


query:Software engineer with experience in Python and machine learning
Lista de documentos relevantes a la query:

Doc: doc_3651 (Similitud: 0.4583)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: machine learning engineerposition: Principal Machine Learning Engineer description: [' Building production-level machine learning models for a variety of clients. Solving complex commercial problems with innovative machine learning solutions. ', ' An MSc or Ph.D. in a quantitative discipline (e.g. Maths, Stats, Computer Science, Physics etc) Experience programming with production-level Python. Experience in building machine learning models and deploying them into production. A passion for seeing your work go into external products and having an impact. '] location: London

Doc: doc_3643 (Similitud: 0.4503)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: machine learning engineerposition: Machine 

In [6]:
nombre_modelo = "Modelo_V2" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Marketing manager skilled in digital marketing and SEO strategies"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)
# Mostrar resultados
print(f'query:{terms}')

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)


query:Marketing manager skilled in digital marketing and SEO strategies
Lista de documentos relevantes a la query:

Doc: doc_4071 (Similitud: 0.5400)
Contenido del documento:
major_job: ADMINISTRATIVE AND COMMERCIAL MANAGERS job: marketing directorposition: Marketing Director description: [' Definition marketing objectives based on global company and sales objectives Work closely with the management and sales to develop the define the marketing strategy and marketing action plan to create awareness, acquire new customers and retain those customers Develop multi-channel marketing initiatives that include Media and PR, Event marketing, content marketing, digital marketing, and one-on-one marketing Manage the creation of brand assets and assure that the brand identity is followed throughout all assets and communication activities also in local markets Be responsible for all aspects of daily management for the marketing function. Coordinate the marketing team, give strategic direction, and

In [7]:
nombre_modelo = "Modelo_V2" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)


# Ingresar la consulta
terms = "Data analyst proficient in Python and data visualization tools"



# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)

Lista de documentos relevantes a la query:

Doc: doc_3520 (Similitud: 0.3594)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: data analystposition: Data Analyst Focus Glossary (M/F) description: [' A Data Analyst to support the continuous improvement and deployment of the Data Governance capabilities at the Bank, in order to enforce our Data Quality & Governance Team and its growing activity. '] location: Luxembourg City

Doc: doc_3514 (Similitud: 0.3155)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: data analystposition: Senior Business Data Analyst description: [' Strong Data Analyst background ETL Relational Datamodelling Good understanding of Data Warehouse solutions Excellent stakeholder management skills ', ' Financial Services background SQL, Python '] location: London

Doc: doc_3508 (Similitud: 0.3001)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY 

## Resultado Modelo 2

In [12]:
# Importar las funciones necesarias
from cargar_modelo import CargarModelo, PreProcesarConsulta, RecuperarDocumentosRelevantes

# Especificar el nombre del modelo
nombre_modelo = "Modelo_V2"  

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Definir las consultas y los documentos relevantes
consultas_relevantes = {
    "Software engineer with experience in Python and machine learning": [
        "doc_3651", 
        "doc_3654"    ],
    
    "Marketing manager skilled in digital marketing and SEO strategies": [
        "doc_4082", 
        "doc_2730"    ],
    
    "Data analyst proficient in Python and data visualization tools": [
        "doc_3514", 
        "doc_3520"    ]
}


# Definir la función calcular_mrr (asegúrate de tenerla en tu código)

def calcular_mrr(consultas_relevantes, vectorizer, tfidf_matrix, doc_id, remove_stopwords, use_lemmatization):
    mrr_total = 0
    num_consultas = len(consultas_relevantes)
    
    for consulta, docs_relevantes in consultas_relevantes.items():
        # Preprocesar y transformar la consulta
        query_preprocesada = PreProcesarConsulta(consulta, remove_stopwords, use_lemmatization)
        query_vector = vectorizer.transform([query_preprocesada])
        
        # Obtener similitudes coseno y rankings
        cosine_similarities = (tfidf_matrix * query_vector.T).toarray().flatten()
        top_indices = cosine_similarities.argsort()[::-1]
        ranked_docs = [(cosine_similarities[idx], doc_id[idx]) for idx in top_indices]
        
        # Encontrar el rank del primer documento relevante
        rank_relevante = None
        for idx, (sim, doc) in enumerate(ranked_docs, start=1):
            if doc in docs_relevantes:
                rank_relevante = idx
                break
        
        if rank_relevante:
            reciprocal_rank = 1 / rank_relevante
            mrr_total += reciprocal_rank
            print(f"Consulta '{consulta}': Documento relevante encontrado en la posición {rank_relevante}.")
        else:
            print(f"Consulta '{consulta}': No se encontró un documento relevante en el ranking.")
    
    # Calcular el MRR
    mrr = mrr_total / num_consultas
    print(f"\nEl MRR calculado es: {mrr:.4f}")
    return mrr

# Calcular el MRR para el modelo cargado
mrr = calcular_mrr(
    consultas_relevantes,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords,
    use_lemmatization
)

Consulta 'Software engineer with experience in Python and machine learning': Documento relevante encontrado en la posición 1.
Consulta 'Marketing manager skilled in digital marketing and SEO strategies': Documento relevante encontrado en la posición 2.
Consulta 'Data analyst proficient in Python and data visualization tools': Documento relevante encontrado en la posición 1.

El MRR calculado es: 0.8333


## Modelo Versión 3:


In [8]:
from cargar_modelo import CargarModelo, PreProcesarConsulta, RecuperarDocumentosRelevantes, MostrarDocumentos

# Especificar el modelo a utilizar
nombre_modelo = "Modelo_V3" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Software engineer with experience in Python and machine learning"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)

# Mostrar resultados
print(f'query:{terms}')

MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)


query:Software engineer with experience in Python and machine learning
Lista de documentos relevantes a la query:

Doc: doc_3651 (Similitud: 0.3024)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: machine learning engineerposition: Principal Machine Learning Engineer description: [' Building production-level machine learning models for a variety of clients. Solving complex commercial problems with innovative machine learning solutions. ', ' An MSc or Ph.D. in a quantitative discipline (e.g. Maths, Stats, Computer Science, Physics etc) Experience programming with production-level Python. Experience in building machine learning models and deploying them into production. A passion for seeing your work go into external products and having an impact. '] location: London

Doc: doc_3605 (Similitud: 0.2381)
Contenido del documento:
major_job: INFORMATION AND COMMUNICATIONS TECHNOLOGY PROFESSIONALS job: data scientistposition: Senior Data Scienti

In [9]:

nombre_modelo = "Modelo_V3" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Marketing manager skilled in digital marketing and SEO strategies"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)
# Mostrar resultados
print(f'query:{terms}')

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)


query:Marketing manager skilled in digital marketing and SEO strategies
Lista de documentos relevantes a la query:

Doc: doc_4082 (Similitud: 0.5344)
Contenido del documento:
major_job: ADMINISTRATIVE AND COMMERCIAL MANAGERS job: marketing directorposition: Digital Marketing Director description: [' Develop and deploy the global and regional digital marketing strategy to support marketing and sales colleagues in the field. Set digital marketing objectives in alignment with the Executive Director, Marketing, and the wider team. Manage all digital marketing channels (e.g. website, CRM, social media, etc.) to ensure consistency. Measure ROI and KPIs; prepare budgets, advise on maximising digital marketing spends across all digital activities, closely monitor campaign results and produce timely reporting to the leadership team. Champion best in class digital marketing practices, inspire digital first thinking, advise on and drive digital marketing initiatives. Work with broader marketing t

In [10]:

nombre_modelo = "Modelo_V3" 

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
elif nombre_modelo == "Modelo_V3":
    remove_stopwords = True
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Ingresar la consulta
terms = "Marketing manager skilled in digital marketing and SEO strategies"

# Recuperar documentos relevantes
DocsRelevantes = RecuperarDocumentosRelevantes(
    terms,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords=remove_stopwords,
    use_lemmatization=use_lemmatization
)
# Mostrar resultados
print(f'query:{terms}')

# Mostrar resultados
MostrarDocumentos(DocsRelevantes, doc_contents, mostrar_contenido=True)

query:Marketing manager skilled in digital marketing and SEO strategies
Lista de documentos relevantes a la query:

Doc: doc_4082 (Similitud: 0.5344)
Contenido del documento:
major_job: ADMINISTRATIVE AND COMMERCIAL MANAGERS job: marketing directorposition: Digital Marketing Director description: [' Develop and deploy the global and regional digital marketing strategy to support marketing and sales colleagues in the field. Set digital marketing objectives in alignment with the Executive Director, Marketing, and the wider team. Manage all digital marketing channels (e.g. website, CRM, social media, etc.) to ensure consistency. Measure ROI and KPIs; prepare budgets, advise on maximising digital marketing spends across all digital activities, closely monitor campaign results and produce timely reporting to the leadership team. Champion best in class digital marketing practices, inspire digital first thinking, advise on and drive digital marketing initiatives. Work with broader marketing t

## Resultado Modelo 3 (Versión Final)

In [13]:
# Importar las funciones necesarias
from cargar_modelo import CargarModelo, PreProcesarConsulta, RecuperarDocumentosRelevantes

# Especificar el nombre del modelo
nombre_modelo = "Modelo_V3"  

# Ajustar los parámetros de preprocesamiento según el modelo
if nombre_modelo == "Modelo_V1":
    remove_stopwords = False
    use_lemmatization = False
elif nombre_modelo == "Modelo_V2":
    remove_stopwords = False
    use_lemmatization = True
else:
    remove_stopwords = True
    use_lemmatization = True

# Cargar el modelo
vectorizer, tfidf_matrix, doc_id, doc_contents = CargarModelo(nombre_modelo)

# Definir las consultas y los documentos relevantes
consultas_relevantes = {
    "Software engineer with experience in Python and machine learning": [
        "doc_3651", 
        "doc_3654"
    ],
    
    "Marketing manager skilled in digital marketing and SEO strategies": [
        "doc_4082", 
        "doc_2730"
    ],
    
    "Data analyst proficient in Python and data visualization tools": [
        "doc_3514", 
        "doc_3520"
    ]
}


# Definir la función calcular_mrr (asegúrate de tenerla en tu código)

def calcular_mrr(consultas_relevantes, vectorizer, tfidf_matrix, doc_id, remove_stopwords, use_lemmatization):
    mrr_total = 0
    num_consultas = len(consultas_relevantes)
    
    for consulta, docs_relevantes in consultas_relevantes.items():
        # Preprocesar y transformar la consulta
        query_preprocesada = PreProcesarConsulta(consulta, remove_stopwords, use_lemmatization)
        query_vector = vectorizer.transform([query_preprocesada])
        
        # Obtener similitudes coseno y rankings
        cosine_similarities = (tfidf_matrix * query_vector.T).toarray().flatten()
        top_indices = cosine_similarities.argsort()[::-1]
        ranked_docs = [(cosine_similarities[idx], doc_id[idx]) for idx in top_indices]
        
        # Encontrar el rank del primer documento relevante
        rank_relevante = None
        for idx, (sim, doc) in enumerate(ranked_docs, start=1):
            if doc in docs_relevantes:
                rank_relevante = idx
                break
        
        if rank_relevante:
            reciprocal_rank = 1 / rank_relevante
            mrr_total += reciprocal_rank
            print(f"Consulta '{consulta}': Documento relevante encontrado en la posición {rank_relevante}.")
        else:
            print(f"Consulta '{consulta}': No se encontró un documento relevante en el ranking.")
    
    # Calcular el MRR
    mrr = mrr_total / num_consultas
    print(f"\nEl MRR calculado es: {mrr:.4f}")
    return mrr

# Calcular el MRR para el modelo cargado
mrr = calcular_mrr(
    consultas_relevantes,
    vectorizer,
    tfidf_matrix,
    doc_id,
    remove_stopwords,
    use_lemmatization
)


Consulta 'Software engineer with experience in Python and machine learning': Documento relevante encontrado en la posición 1.
Consulta 'Marketing manager skilled in digital marketing and SEO strategies': Documento relevante encontrado en la posición 1.
Consulta 'Data analyst proficient in Python and data visualization tools': Documento relevante encontrado en la posición 1.

El MRR calculado es: 1.0000


# Anexo 

## Justificación de la relevancia de los documentos seleccionados frente a otros

### 1. **Software engineer with experience in Python and machine learning**
- **doc_3651**:
  - **Descripción**: *Principal Machine Learning Engineer* con experiencia en Python avanzado y despliegue de modelos en producción.
  - **Relevancia frente a otros**: Este documento destaca porque especifica una experiencia sólida en producción de modelos de *machine learning*, una habilidad esencial que otros documentos no enfatizan tanto. Además, requiere habilidades avanzadas en Python, lo que lo hace muy relevante frente a otros documentos que no profundizan en la parte productiva del *machine learning*.

- **doc_3654**:
  - **Descripción**: Ingeniero especializado en diseñar e implementar sistemas de *machine learning* usando TensorFlow o PyTorch con Python.
  - **Relevancia frente a otros**: A diferencia de otros documentos, este se enfoca claramente en el diseño de sistemas de *machine learning* y en la implementación práctica, lo que lo hace más relevante que otros que se centran en teorías o proyectos más específicos.

### 2. **Marketing manager skilled in digital marketing and SEO strategies**
- **doc_4082**:
  - **Descripción**: *Digital Marketing Director* responsable de estrategias globales de marketing digital, con fuerte enfoque en SEO y KPIs.
  - **Relevancia frente a otros**: Destaca frente a otros documentos por su enfoque integral en marketing digital global, liderazgo de equipos, y estrategias avanzadas de SEO, lo que supera otras posiciones que tienen una visión más operativa o regional.

- **doc_2730**:
  - **Descripción**: *Digital Marketer* con experiencia en SEO y creación de contenido.
  - **Relevancia frente a otros**: Este documento es más relevante que otros por su enfoque en SEO, creación de contenido, y marketing en redes sociales, que son habilidades específicas solicitadas. Otros documentos no abarcan tanto estas áreas clave.

### 3. **Data analyst proficient in Python and data visualization tools**
- **doc_3514**:
  - **Descripción**: *Senior Business Data Analyst* con fuerte manejo de Python y modelado de datos.
  - **Relevancia frente a otros**: Se destaca por la combinación de experiencia en Python y habilidades de modelado de datos, lo que lo hace más relevante frente a otros documentos que no enfatizan tanto estas habilidades técnicas avanzadas.

- **doc_3520**:
  - **Descripción**: *Data Analyst* enfocado en gobernanza y calidad de datos, con habilidades de visualización.
  - **Relevancia frente a otros**: Aunque no menciona Python de forma destacada, su enfoque en la gobernanza y la calidad de los datos lo hace más relevante que otros documentos, que no cubren aspectos importantes como la integridad de los datos y la visualización en profundidad.
